In [1]:
# 필요한 라이브러리 설치
!pip install torch konlpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from konlpy.tag import Okt
import numpy as np
from collections import Counter, defaultdict

# 형용사가 추가된 확장된 샘플 데이터
sentences = [
    # 기존 문장들
    "나는 학교에 간다",
    "그는 책을 읽는다",
    "우리는 공부를 한다",
    "그녀는 음악을 듣는다",
    "아이들이 놀고 있다",
    # 형용사가 포함된 새로운 문장들
    "예쁜 꽃이 핀다",
    "큰 개가 뛴다",
    "작은 새가 난다",
    "맛있는 음식을 먹는다",
    "재미있는 영화를 본다",
    "빠른 자동차가 온다",
    "높은 건물이 보인다",
    "따뜻한 날씨가 좋다"
]

pos_tags = [
    # 기존 품사 태그들
    ["Noun", "Noun", "Verb"],
    ["Noun", "Noun", "Verb"],
    ["Noun", "Noun", "Verb"],
    ["Noun", "Noun", "Verb"],
    ["Noun", "Verb", "Verb"],
    # 형용사 포함 새로운 품사 태그들
    ["Adjective", "Noun", "Verb"],
    ["Adjective", "Noun", "Verb"],
    ["Adjective", "Noun", "Verb"],
    ["Adjective", "Noun", "Verb"],
    ["Adjective", "Noun", "Verb"],
    ["Adjective", "Noun", "Verb"],
    ["Adjective", "Noun", "Verb"],
    ["Adjective", "Noun", "Verb"]
]

# 확장된 어휘 사전 구축
word_to_idx = {"<PAD>": 0, "<UNK>": 1}
tag_to_idx = {"<PAD>": 0}

# 단어 사전 구축
for sentence in sentences:
    for word in sentence.split():
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)

# 품사 사전 구축 (이제 Adjective 포함)
for tags in pos_tags:
    for tag in tags:
        if tag not in tag_to_idx:
            tag_to_idx[tag] = len(tag_to_idx)

idx_to_tag = {idx: tag for tag, idx in tag_to_idx.items()}

# 사전 정보 출력
print("=== 사전 정보 ===")
print(f"단어 사전 크기: {len(word_to_idx)}")
print(f"품사 사전: {tag_to_idx}")
print(f"총 품사 개수: {len(tag_to_idx)}")
print()

# BiLSTM 모델 정의 (동일)
class BiLSTM_POS(nn.Module):
    def __init__(self, vocab_size, tagset_size, embedding_dim, hidden_dim):
        super(BiLSTM_POS, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embedings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, bidirectional=True, batch_first=True)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embedings(sentence)
        lstm_out, _ = self.lstm(embeds)
        tag_space = self.hidden2tag(lstm_out)
        return tag_space

# 데이터셋 클래스 (동일)
class POSDataset(Dataset):
    def __init__(self, sentences, tags, word_to_idx, tag_to_idx):
        self.sentences = sentences
        self.tags = tags
        self.word_to_idx = word_to_idx
        self.tag_to_idx = tag_to_idx

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx].split()
        tag_seq = self.tags[idx]

        # 단어를 인덱스로 변환
        word_indices = [self.word_to_idx.get(word, self.word_to_idx["<UNK>"]) for word in sentence]
        tag_indices = [self.tag_to_idx[tag] for tag in tag_seq]

        return torch.LongTensor(word_indices), torch.LongTensor(tag_indices)

# 확장된 데이터로 데이터 로더 생성
dataset = POSDataset(sentences, pos_tags, word_to_idx, tag_to_idx)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# 모델 초기화 (품사 개수가 증가했으므로 재초기화 필요)
EMBEDDING_DIM = 50
HIDDEN_DIM = 100
model = BiLSTM_POS(len(word_to_idx), len(tag_to_idx), EMBEDDING_DIM, HIDDEN_DIM)
loss_function = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.SGD(model.parameters(), lr=0.1)

print("=== 모델 훈련 시작 ===")
print(f"총 훈련 문장 수: {len(sentences)}")
print()

# 훈련 (에포크를 좀 더 늘려서 형용사도 잘 학습하도록)
for epoch in range(150):
    total_loss = 0
    for sentence, tags in dataloader:
        model.zero_grad()
        tag_scores = model(sentence)
        loss = loss_function(tag_scores.view(-1, len(tag_to_idx)), tags.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if epoch % 30 == 0:
        print(f'Epoch {epoch}, Loss: {total_loss:.4f}')

print("\n=== 훈련 완료 ===\n")

# 예측 함수 (동일)
def predict_pos(sentence):
    words = sentence.split()
    word_indices = [word_to_idx.get(word, word_to_idx["<UNK>"]) for word in words]
    inputs = torch.LongTensor(word_indices).unsqueeze(0)

    with torch.no_grad():
        tag_scores = model(inputs)
        predicted_tags = torch.argmax(tag_scores, dim=2)

    result = []
    for word, tag_idx in zip(words, predicted_tags[0]):
        result.append((word, idx_to_tag[tag_idx.item()]))

    return result

# 다양한 테스트 문장들 (형용사 포함)
test_sentences = [
    "고양이가 물고기를 먹는다",        # 기존 테스트
    "빨간 사과가 달다",               # 형용사 + 새로운 단어
    "높은 산이 보인다",               # 형용사 포함
    "작은 아이가 뛴다",               # 학습된 형용사
    "어려운 문제를 푼다",             # 새로운 형용사
    "그는 빠르게 달린다"              # 부사 vs 형용사 구분 테스트
]

print("=== 테스트 결과 ===")
for test_sentence in test_sentences:
    result = predict_pos(test_sentence)
    print(f"입력: {test_sentence}")
    print(f"결과: {result}")

    # 더 보기 좋게 출력
    formatted_result = " ".join([f"{word}({tag})" for word, tag in result])
    print(f"형태: {formatted_result}")
    print("-" * 50)

# 훈련 데이터에서 형용사가 잘 인식되는지 확인
print("\n=== 훈련 데이터 검증 ===")
for i, sentence in enumerate(sentences[5:8]):  # 형용사 포함 문장들만
    result = predict_pos(sentence)
    actual_tags = pos_tags[i+5]

    print(f"문장: {sentence}")
    print(f"실제: {actual_tags}")
    print(f"예측: {[tag for _, tag in result]}")

    # 정확도 계산
    correct = sum(1 for (_, pred_tag), actual_tag in zip(result, actual_tags) if pred_tag == actual_tag)
    accuracy = correct / len(actual_tags) * 100
    print(f"정확도: {accuracy:.1f}%")
    print("-" * 30)

# 품사별 통계
print("\n=== 품사별 통계 ===")
tag_counts = Counter()
for tags in pos_tags:
    for tag in tags:
        tag_counts[tag] += 1

for tag, count in tag_counts.items():
    if tag != "<PAD>":
        print(f"{tag}: {count}개")

=== 사전 정보 ===
단어 사전 크기: 41
품사 사전: {'<PAD>': 0, 'Noun': 1, 'Verb': 2, 'Adjective': 3}
총 품사 개수: 4

=== 모델 훈련 시작 ===
총 훈련 문장 수: 13

Epoch 0, Loss: 16.9447
Epoch 30, Loss: 0.5474
Epoch 60, Loss: 0.1511
Epoch 90, Loss: 0.0804
Epoch 120, Loss: 0.0531

=== 훈련 완료 ===

=== 테스트 결과 ===
입력: 고양이가 물고기를 먹는다
결과: [('고양이가', 'Noun'), ('물고기를', 'Verb'), ('먹는다', 'Verb')]
형태: 고양이가(Noun) 물고기를(Verb) 먹는다(Verb)
--------------------------------------------------
입력: 빨간 사과가 달다
결과: [('빨간', 'Noun'), ('사과가', 'Noun'), ('달다', 'Noun')]
형태: 빨간(Noun) 사과가(Noun) 달다(Noun)
--------------------------------------------------
입력: 높은 산이 보인다
결과: [('높은', 'Adjective'), ('산이', 'Noun'), ('보인다', 'Verb')]
형태: 높은(Adjective) 산이(Noun) 보인다(Verb)
--------------------------------------------------
입력: 작은 아이가 뛴다
결과: [('작은', 'Adjective'), ('아이가', 'Noun'), ('뛴다', 'Verb')]
형태: 작은(Adjective) 아이가(Noun) 뛴다(Verb)
--------------------------------------------------
입력: 어려운 문제를 푼다
결과: [('어려운', 'Noun'), ('문제를', 'Noun'), ('푼다', 'Noun')]
형태: 어려운(Noun) 문제를(